<a href="https://colab.research.google.com/github/YarkoMarko/nn_2/blob/main/module4/%D0%94%D0%97_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [1]:
import pandas as pd

Дані про банани

In [2]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/banana_quality.csv')

df['Quality'] = df['Quality'].map({'Good': 1, 'Bad': 0})

columns = df.columns.delete(-1)
data = df[columns]
df[columns] = (data - data.min()) / (data.max() - data.min())

In [3]:
df.head()

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,0.380309,0.626747,0.680712,0.360975,0.567312,0.671933,0.543416,1
1,0.349951,0.628093,0.485275,0.293682,0.481689,0.646850,0.545720,1
2,0.478460,0.699448,0.572694,0.283811,0.499358,0.716580,0.617337,1
3,0.446465,0.705394,0.595677,0.374028,0.473461,0.633590,0.556625,1
4,0.541672,0.687704,0.458841,0.378242,0.442847,0.579430,0.705908,1


Потрібно на основі інформації про банан спрогнозувати його якість `Quality`

# Завдання 1
Створіть клас Dataset для тренувальних та тестових даних

In [9]:
import torch
from torch.utils.data import Dataset

device = "cpu"

class UsedDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X.values).to(device)
    self.y = torch.FloatTensor(y.values).to(device)
    self.y = self.y.reshape(-1, 1)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    return self.X[index], self.y[index]

y = df["Quality"]
X = df.drop(columns=["Quality"])

dataset = UsedDataset(X, y)

In [6]:
X.shape

(8000, 7)

In [7]:
y.shape

(8000,)

# Завдання 2
Створіть DataLoader

In [10]:
from torch.utils.data import DataLoader

batch = 250
dataloader = DataLoader(dataset, batch_size=batch, shuffle=True, generator=torch.Generator(device=device))

In [13]:
X_batch, y_batch = next(iter(dataloader))

# Завдання 3
Створіть нейромережу

In [11]:
from torch import nn

model = nn.Sequential(
    nn.Linear(7, 14),
    nn.LeakyReLU(),
    nn.Linear(14, 14),
    nn.LeakyReLU(),
    nn.Linear(14, 1)
)

# Завдання 4
Визначте функцію втрат та оптимізатор

In [12]:
loss_fn = nn.MSELoss()

optim = torch.optim.SGD(model.parameters(), lr=0.003)

# Завдання 5
Натренуйте нейромережу. Візміть невелику кількість епох

In [14]:
num_epochs = 7

for epoch in range(num_epochs):
  for X_batch, y_batch in dataloader:
    y_pred = model(X_batch)
    loss = loss_fn(y_pred, y_batch)

    loss.backward()
    optim.step()
    optim.zero_grad()

# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [15]:
torch.save(model, "model.pt")